## • create a connection to the database using the library sqlite3

In [159]:
import sqlite3
from pandas.io.sql import read_sql

con = sqlite3.connect('../data/checking-logs.sqlite')
con

## get the schema of the table test


In [160]:
read_sql('PRAGMA table_info(test)', con)


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


## get only 10 first rows of the table test to check how the table looks like

In [161]:
test = read_sql('SELECT * FROM test LIMIT 10', con)
test

,index,uid,labname,first_commit_ts,first_view_ts
0,0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


## find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query
* do it by joining the table with the table deadlines
* the difference should be displayed in hours
* do not take into account the lab ’project1’, it has longer deadlines and will be
an outlier
* the value should be stored in the dataframe df_min with the corresponding
uid


In [162]:
query = '''
    SELECT uid, min(CAST((julianday(deadlines, 'unixepoch') - julianday(first_commit_ts)) * 24 AS INT)) as delta from test
    LEFT JOIN deadlines on deadlines.labs = test.labname
    WHERE labname NOT IN ('project1')
'''
df_min = read_sql(query, con)
df_min

,uid,delta
0,user_25,2


## do the same thing but with the maximum using only one query, the dataframe name is df_max

In [163]:
query = '''
    SELECT uid, max(CAST((julianday(deadlines, 'unixepoch') - julianday(first_commit_ts)) * 24 AS INT)) as delta from test
    LEFT JOIN deadlines on deadlines.labs = test.labname
    WHERE labname NOT IN ('project1')
'''
df_max = read_sql(query, con)
df_max

,uid,delta
0,user_30,202


## do the same thing but with the average using only one query, this time your dataframe should not include the uid column, the dataframe name is df_avg

In [164]:
query = '''
    SELECT avg(CAST((julianday(deadlines, 'unixepoch') - julianday(first_commit_ts)) * 24 AS INT)) as delta from test
    LEFT JOIN deadlines on deadlines.labs = test.labname
    WHERE labname NOT IN ('project1')
'''
df_avg = read_sql(query, con)
df_avg

,delta
0,89.125


## we want to test the hypothesis that the users who visited Newsfeed just a few times have the lower delta between the first commit and the deadline, to do this you need to calculate the  orrelation coefficient between the number of the pageviews and the difference
* using only one query create a table with the columns: uid, avg_diff, pageviews
* uid is the uids that exist in the test
* avg_diff is the average delta between the first commit and the lab deadline per user
* pageviews is the number of Newsfeed visits per user
* do not take into account the lab ’project1’
* store it to the dataframe views_diff
* use the Pandas method corr() to calculate the correlation coefficient between the number of the pageviews and the difference

По сабжу

In [165]:
query = '''
    SELECT test.uid, avg(CAST((julianday(deadlines, 'unixepoch') - julianday(first_commit_ts)) * 24 AS INT)) as avg_diff, count(pageviews.uid)as pageviews  FROM test
    LEFT JOIN deadlines on deadlines.labs = test.labname
    LEFT JOIN pageviews on pageviews.uid = test.uid
    WHERE labname NOT IN ('project1')
    GROUP BY test.uid
'''
views_diff = read_sql(query, con)
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,0.185834
pageviews,0.185834,1.000000


дикость по чек-листу

In [166]:
query = '''
    SELECT test.uid, avg(CAST((julianday(deadlines, 'unixepoch') - julianday(first_commit_ts)) * 24 AS INT)) as avg_diff, count(pageviews.uid)as pageviews  FROM 
    (SELECT uid, labname, first_commit_ts FROM test GROUP BY test.uid) test
    LEFT JOIN deadlines on deadlines.labs = test.labname
    LEFT JOIN pageviews on pageviews.uid = test.uid
    WHERE labname NOT IN ('project1')
    GROUP BY test.uid
'''
views_diff = read_sql(query, con)
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,0.062967
pageviews,0.062967,1.000000


## close the connection


In [167]:
con.close()